## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
import scipy.sparse
#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [3]:
# Create y_df with the Attrition and Department columns

y_df = attrition_df[['Attrition', 'Department']]

In [4]:
# Create a list of at least 10 column names to use as X data
selected_columns = ['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 'JobInvolvement',
                    'JobSatisfaction', 'NumCompaniesWorked', 'WorkLifeBalance', 'YearsAtCompany',
                    'YearsSinceLastPromotion', 'OverTime']

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
print(X_df.dtypes)


Age                         int64
DistanceFromHome            int64
Education                   int64
EnvironmentSatisfaction     int64
JobInvolvement              int64
JobSatisfaction             int64
NumCompaniesWorked          int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
OverTime                   object
dtype: object


In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)


In [6]:
# Convert your X data to numeric data types however you see fit
# Convert 'OverTime' to numeric
X_train['OverTime'] = X_train['OverTime'].map({'Yes': 1, 'No': 0})
X_test['OverTime'] = X_test['OverTime'].map({'Yes': 1, 'No': 0})

# Verify the conversion
print(X_train['OverTime'].value_counts())

# Check data types again
print(X_train.dtypes)

OverTime
0    837
1    339
Name: count, dtype: int64
Age                        int64
DistanceFromHome           int64
Education                  int64
EnvironmentSatisfaction    int64
JobInvolvement             int64
JobSatisfaction            int64
NumCompaniesWorked         int64
WorkLifeBalance            int64
YearsAtCompany             int64
YearsSinceLastPromotion    int64
OverTime                   int64
dtype: object


In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Convert to float type first
X_train = X_train.astype(float)
X_test = X_test.astype(float)

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
# Create a OneHotEncoder for the Department column
try:
    department_encoder = OneHotEncoder(sparse=False)
except TypeError:
    department_encoder = OneHotEncoder()

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department = department_encoder.transform(y_train[['Department']])
y_test_department = department_encoder.transform(y_test[['Department']])

# If using an older version where sparse output is default
if scipy.sparse.issparse(y_train_department):
    y_train_department = y_train_department.toarray()
    y_test_department = y_test_department.toarray()

In [9]:
# Create a OneHotEncoder for the Attrition column
try:
    attrition_encoder = OneHotEncoder(sparse=False)
except TypeError:
    attrition_encoder = OneHotEncoder()

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition = attrition_encoder.transform(y_train[['Attrition']])
y_test_attrition = attrition_encoder.transform(y_test[['Attrition']])

# If using an older version where sparse output is default
if scipy.sparse.issparse(y_train_attrition):
    y_train_attrition = y_train_attrition.toarray()
    y_test_attrition = y_test_attrition.toarray()

## Create, Compile, and Train the Model

In [10]:
# Find the number of columns in the X training data
input_dim = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(input_dim,))

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)


In [11]:
# Create a branch for Department
# Create the hidden layer
department_hidden = layers.Dense(16, activation='relu')(shared_layer2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_hidden)



In [12]:
# Create a branch for Attrition
# Create the hidden layer
attrition_hidden = layers.Dense(16, activation='relu')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(2, activation='softmax', name='attrition_output')(attrition_hidden)


In [13]:
#  Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy',
                       'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 11)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │            768 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 32)             │          2,080 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 16)             │            528 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 16)             │            528 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             51 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             34 │ dense_3[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,989 (15.58 KB)

 Trainable params: 3,989 (15.58 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Train the model
history = model.fit(X_train_scaled,
                    {'department_output': y_train_department,
                     'attrition_output': y_train_attrition},
                    epochs=100,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=1)

Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - attrition_output_accuracy: 0.7589 - department_output_accuracy: 0.1943 - loss: 1.8769 - val_attrition_output_accuracy: 0.7966 - val_department_output_accuracy: 0.5805 - val_loss: 1.4873
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8253 - department_output_accuracy: 0.6492 - loss: 1.3855 - val_attrition_output_accuracy: 0.7966 - val_department_output_accuracy: 0.6356 - val_loss: 1.3207
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8247 - department_output_accuracy: 0.6503 - loss: 1.2422 - val_attrition_output_accuracy: 0.7966 - val_department_output_accuracy: 0.6314 - val_loss: 1.2620
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8309 - department_output_accuracy: 0.6508 - loss: 1.1772 - val_attrition_output_accuracy: 0.7966 - val_department_output_accuracy: 0.6314 - val_loss: 1.2334
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [15]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled,
                              {'department_output': y_test_department,
                               'attrition_output': y_test_attrition},
                              verbose=0)
# Print the entire test_results
print("Test results:", test_results)

Test results: [2.4832723140716553, 0.8333333134651184, 0.5340136289596558]


In [16]:
# Print the accuracy for both department and attrition
if len(test_results) >= 5:
    print(f"Department predictions accuracy: {test_results[3]:.4f}")
    print(f"Attrition predictions accuracy: {test_results[4]:.4f}")
elif len(test_results) >= 3:
    print(f"Department predictions accuracy: {test_results[1]:.4f}")
    print(f"Attrition predictions accuracy: {test_results[2]:.4f}")
else:
    print("Unable to determine accuracies. Test results structure:", test_results)

Department predictions accuracy: 0.8333
Attrition predictions accuracy: 0.5340


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy isn't great for this data. Attrition prediction is probably imbalanced, so accuracy can be misleading. Other metrics like precision, recall, or F1-score might be better. They show how good the model is at finding employees who might leave.
2.We used softmax for both outputs. It's good for Department prediction cause there's multiple categories. For Attrition, sigmoid might be more common for binary stuff, but softmax works too. Softmax makes the outputs add up to 1, which is nice.
3. Yeah, we could:
Make better features,
Fix class imbalance,
Try different model settings,
Add more layers maybe,
Use other models like random forests too,
Do cross-validation,
Use regularization to stop overfitting,

These might help, especially for attrition prediction which isn't great right now.